##### This file is set for testing the performance of different algorithms based according to the LP benchmark.

In [57]:
import numpy as np
import pandas as pd
import time
from tabulate import tabulate

class OnlineMatchingTest:
    def __init__(self, N, T, E, algorithm_class, runs, inventory_levels, rewards=None, usage_duration_distributions=None):
        self.N = N
        self.T = T
        self.E = E
        self.algorithm_class = algorithm_class
        self.runs = runs
        self.rewards = rewards if rewards else np.linspace(15, 30, N)
        self.usage_duration_distributions = usage_duration_distributions if usage_duration_distributions else {n: np.random.choice(range(3, 6), size=2, replace=False).tolist() for n in range(N)}
        self.inventory_levels = inventory_levels
        
    def run_algorithm(self):
        results = []
        for _ in range(self.runs):
            model = self.algorithm_class(self.N, self.T, self.E, self.rewards, self.usage_duration_distributions, self.inventory_levels)
            start_time = time.time()
            total_reward = 0
            for t in range(self.T):
                result = model.allocate_resource(t)
                if result:
                    resource, _ = result
                    total_reward += self.rewards[resource]
            elapsed_time = time.time() - start_time
            results.append((total_reward, elapsed_time))
        return results
    
    def compute_statistics(self):
        results = self.run_algorithm()
        total_rewards = [result[0] for result in results]
        elapsed_times = [result[1] for result in results]

        data = {
            'Algorithm': self.algorithm_class.__name__,
            'Mean Reward': [np.mean(total_rewards)],
            'Standard Deviation Reward': [np.std(total_rewards)],
            'Mean Time': [np.mean(elapsed_times)],
            'Standard Deviation Time': [np.std(elapsed_times)]
        }
        return pd.DataFrame(data)

##### Benchmark

In [58]:
import cvxpy as cp
import numpy as np
import pandas as pd

# Parameters
n = 5  # Number of products
T = 50  # Number of periods
rewards = np.linspace(15, 30, n)  # Prices of products
inventory_levels = [1, 5, 20]  # Inventory levels
probabilities = np.array([1 / (20 - i) for i in range(1, 6)])  # Geometric distribution parameters
survival_probabilities = np.array([(1 - probabilities[i]) ** np.arange(T) for i in range(n)])  # Survival probabilities

# To store results for statistical analysis
results = {level: [] for level in inventory_levels}

# Run the simulation 100 times
runs = 100
for _ in range(runs):
    for inventory_level in inventory_levels:
        X = cp.Variable((T, n), nonneg=True)
        revenue = cp.sum(cp.multiply(X, rewards.reshape(1, n)))
        constraints = [
            cp.sum(X, axis=1) <= 1,  # Constraint 1
            X <= 1  # Constraint 3: X[t, i] must be between 0 and 1
        ]

        # Constraint 2: Capacity constraints for each product
        for i in range(n):
            constraints.append(cp.sum(cp.multiply(X[:, i], survival_probabilities[i])) <= inventory_level)

        # Assuming all items are always available
        l_ti = np.ones((T, n))
        constraints.append(X <= l_ti)  # Constraint 4

        # Define and solve the optimization problem
        prob = cp.Problem(cp.Maximize(revenue), constraints)
        prob.solve()

        # Store the maximum revenue result
        results[inventory_level].append(prob.value)

# Calculating mean and standard deviation of the results
summary = pd.DataFrame({
    level: {
        'Mean Revenue': np.mean(results[level]),
        'STD Revenue': np.std(results[level])
    }
    for level in inventory_levels
}).T

print(summary)


    Mean Revenue   STD Revenue
1     865.128775  2.273737e-13
5    1424.622753  2.273737e-13
20   1500.000000  6.821210e-13


##### Rank Based Allocation Algorithm

In [59]:
import numpy as np
import cvxpy as cp

class RankBasedAlgorithm:
    def __init__(self, N, T, E, rewards, usage_duration_distributions, inventory_levels):
        self.N = N  # Number of resources
        self.T = T  # Number of time steps or requests
        self.E = E  # Edges representing possible matches between resources and requests
        self.rewards = rewards  # Rewards for allocating each resource
        self.usage_duration_distributions = usage_duration_distributions  # Usage durations for each resource

        # Initialize resource units' data structures
        self.unit_availability = {}
        self.unit_rank = {}
        self.unit_return_time = {}
        self.unit_usage_count = {}
        self.inventory_levels = {}

        # Assign inventory levels cyclically across resources
        level_count = len(inventory_levels)
        for n in range(N):
            num_units = len(self.usage_duration_distributions[n])
            self.unit_availability[n] = [True] * num_units
            self.unit_rank[n] = list(range(num_units))
            self.unit_return_time[n] = [-1] * num_units
            self.unit_usage_count[n] = [0] * num_units
            self.inventory_levels[n] = inventory_levels[n % level_count]  # Cyclic assignment of inventory levels

    def g(self, x):
        return np.exp(-x)
    
    def update_availability(self, t):
        for i in range(self.N):
            for k in range(len(self.unit_availability[i])):
                if t >= self.unit_return_time[i][k]:
                    self.unit_availability[i][k] = True

    def allocate_resource(self, t):
        self.update_availability(t)
        scores = {}

        for i in self.E[t]:
            available_units = [k for k, available in enumerate(self.unit_availability[i]) 
                               if available and self.unit_usage_count[i][k] < self.inventory_levels[i]]
            if available_units:
                highest_ranked_unit = max(available_units, key=lambda x: self.unit_rank[i][x])
                score = self.rewards[i] * (1 - self.g(self.unit_rank[i][highest_ranked_unit] / len(self.unit_rank[i])))
                scores[i] = score

        if scores:
            selected_resource = max(scores, key=scores.get)
            selected_unit = max([k for k, available in enumerate(self.unit_availability[selected_resource]) 
                                 if available], key=lambda x: self.unit_rank[selected_resource][x])
            self.unit_availability[selected_resource][selected_unit] = False
            duration = np.random.choice(self.usage_duration_distributions[selected_resource])
            self.unit_return_time[selected_resource][selected_unit] = t + duration
            self.unit_usage_count[selected_resource][selected_unit] += 1  # Increment usage count
            return selected_resource, selected_unit

        return None, None


##### Inventory Balancing Algorithm

In [60]:
class InventoryBalancingAlgorithm:
    def __init__(self, N, T, E, rewards, usage_duration_distributions, inventory_levels):
        self.N = N  # Number of offline vertices (resources)
        self.T = T  # Number of online vertices (requests)
        self.E = E  # Edges in the bipartite graph
        self.rewards = rewards  # Reward for each resource
        self.usage_duration_distributions = usage_duration_distributions  # Usage duration distributions
        self.inventory_levels = inventory_levels  # Maximum allocations allowed for each resource
        
        # Initialize inventory, return times, and usage count for each resource
        self.inventory = [1] * self.N  # All resources start as available
        self.return_times = [[] for _ in range(self.N)]   # Return times for each resource
        self.usage_count = [0] * self.N  # Usage count for each resource

    def g(self, x):
        # Adjustment function used to modify resource selection priority
        return np.exp(-x)

    def update_inventory(self, current_time):
        # Updates the availability of resources based on their return times and usage limits
        for n in range(self.N):
            self.return_times[n] = [time for time in self.return_times[n] if time > current_time]
            self.inventory[n] = 1 if len(self.return_times[n]) == 0 and self.usage_count[n] < self.inventory_levels[n] else 0

    def allocate_resource(self, t):
        # Update inventory based on the current time
        self.update_inventory(t)
        
        # Calculate scores for each available resource at time t
        scores = {}
        for n in self.E[t]:
            if self.inventory[n]:  # Only consider available resources
                score = self.rewards[n] * (1 - self.g(self.inventory[n]))
                scores[n] = score
        
        if scores:
            # Select the resource with the highest score
            selected_resource = max(scores, key=scores.get)
            
            # Determine the duration the resource will be unavailable
            duration = np.random.choice(self.usage_duration_distributions[selected_resource])
            self.return_times[selected_resource].append(t + duration)
            
            # Increment usage count and mark the resource as unavailable if limit reached
            self.usage_count[selected_resource] += 1
            if self.usage_count[selected_resource] >= self.inventory_levels[selected_resource]:
                self.inventory[selected_resource] = 0  # Mark as permanently unavailable if max usage reached
            
            return (selected_resource, None)
        else:
            # If no resources are available, return None
            return None


##### Greedy Algorithm

In [61]:
import numpy as np

class GreedyAlgorithm1:
    def __init__(self, N, T, E, rewards, usage_duration_distributions, inventory_levels):
        self.N = N  # Number of offline vertices (resources)
        self.T = T  # Number of online vertices (requests)
        self.E = E  # Edges in the bipartite graph
        self.rewards = rewards  # Reward for each resource
        self.usage_duration_distributions = usage_duration_distributions  # Usage duration distributions
        self.inventory_levels = inventory_levels  # Maximum allocations allowed for each resource
        
        # Initialize inventory, return times, and usage count for each resource
        self.inventory = [1] * self.N  # All resources start as available
        self.return_times = [[] for _ in range(self.N)]  # Return times for each resource
        self.usage_count = [0] * self.N  # Usage count for each resource

        # Initialize resource availability and return times
        self.unit_availability = [True] * self.N
        self.unit_return_time = [-1] * self.N  # -1 indicates the resource is available

    def update_availability(self, t):
        # Update the availability of resources based on the current time
        for i in range(self.N):
            if self.unit_return_time[i] <= t:
                self.unit_availability[i] = True

    def allocate_resource(self, t):
        # Update resource availability at the current time step
        self.update_availability(t)
        
        # Determine available resources and their corresponding rewards
        available_rewards = [(i, self.rewards[i]) for i in self.E[t] if self.unit_availability[i]]
        
        # Proceed with allocation if there are available resources
        if available_rewards:
            # Select the resource with the highest reward
            selected_resource, _ = max(available_rewards, key=lambda x: x[1])
            selected_unit = None  # In this simpler model, we are not distinguishing between units
            
            # Mark the selected resource as unavailable
            self.unit_availability[selected_resource] = False
            # Set the return time based on the usage duration
            duration = np.random.choice(self.usage_duration_distributions[selected_resource])
            self.unit_return_time[selected_resource] = t + duration
            
            return (selected_resource, selected_unit)
        else:
            return None  # No resources are available to allocate

In [62]:
class GreedyAlgorithm:
    def __init__(self, N, T, E, rewards, usage_duration_distributions, inventory_levels):
        self.N = N  # Number of resources
        self.T = T  # Number of time steps or requests
        self.E = E  # Edges representing possible matches between resources and requests
        self.rewards = rewards  # Rewards for allocating each resource
        self.usage_duration_distributions = usage_duration_distributions  # Usage durations for each resource
        self.inventory_levels = inventory_levels  # Maximum allocation counts for each resource
        
        # Initialize resource availability, return times, and usage counts
        self.unit_availability = [True] * self.N
        self.unit_return_time = [-1] * self.N  # -1 indicates the resource is available
        self.unit_usage_count = [0] * self.N  # Track how many times each resource has been allocated

    def update_availability(self, t):
        # Update the availability of resources based on the current time
        for i in range(self.N):
            if self.unit_return_time[i] <= t:
                self.unit_availability[i] = True

    def allocate_resource(self, t):
        # Update resource availability at the current time step
        self.update_availability(t)
        
        # Determine available resources that have not exceeded their inventory limits
        available_rewards = [(i, self.rewards[i]) for i in self.E[t] 
                             if self.unit_availability[i] and self.unit_usage_count[i] < self.inventory_levels[i]]
        
        # Proceed with allocation if there are available resources
        if available_rewards:
            # Select the resource with the highest reward
            selected_resource, _ = max(available_rewards, key=lambda x: x[1])
            
            # Mark the selected resource as unavailable and increment its usage count
            self.unit_availability[selected_resource] = False
            self.unit_usage_count[selected_resource] += 1  # Increment the usage count for the resource
            
            # Set the return time based on the usage duration
            duration = np.random.choice(self.usage_duration_distributions[selected_resource])
            self.unit_return_time[selected_resource] = t + duration
            
            return (selected_resource, None)  # There are no distinct units in this model
        else:
            return None  # No resources are available to allocate


##### Testing

In [69]:
# Collect and display results from all algorithms
results_frames = []
algorithms = [RankBasedAlgorithm, InventoryBalancingAlgorithm, GreedyAlgorithm]
inventory_levels = [1, 5, 20]  # Example uniform inventory levels for simplicity
N = 5 
E = {t: np.random.choice(N, size=np.random.randint(1, N+1), replace=False).tolist() for t in range(50)}

for algorithm_class in algorithms:
    test_env = OnlineMatchingTest(N=N,
                                T=50, 
                                E=E,
                                algorithm_class=algorithm_class, 
                                runs=100, 
                                inventory_levels=inventory_levels)
    df = test_env.compute_statistics()
    df['Algorithm'] = algorithm_class.__name__
    results_frames.append(df)

# Concatenate all results into a single DataFrame
final_results = pd.concat(results_frames, ignore_index=True)

# Print the results using tabulate for a cleaner table format
print(tabulate(final_results, headers='keys', tablefmt='psql', showindex=False))

TypeError: 'int' object is not iterable